In [ ]:
import optuna

data_file = r"C:\Users\A00836756\PycharmProjects\Micro_HH\MG63_2f1_scaled_data.csv"

init_data(data_file)

# Rango min y max para [E0, E1, tau, alpha]
min_range = [0.0001, 0.0001, 0.001, 0.0001]
max_range = [1.5, 1.5, 1.5, 1.0]

fun = P1(
    variable_num=4,
    min_search_range=min_range,
    max_search_range=max_range
)
prob = fun.get_formatted_problem()

num_agents = 101
num_iterations = 50
num_replicas = 20

def objective(trial):
    gravity = trial.suggest_float("gravity", 0.0001, 1.0, log=True)  
    alpha = trial.suggest_float("alpha_gravity", 0.0001, 1.0, log=True)  
    beta = trial.suggest_float("beta", 1.0001, 1.9999)  
    probability = trial.suggest_float("probability", 0.0001, 0.9999)  
    scale = trial.suggest_float("scale", 0.0001, 0.9999)  

    heur = [
        ('central_force_dynamic', {'gravity': gravity, 'alpha': alpha, 'beta': beta, 'dt': 1.0}, 'all'),
        ('local_random_walk', {'probability': probability, 'scale': scale, 'distribution': 'gaussian'}, 'all')
    ]

    _, perf, _ = evaluate_sequence_performance_all(heur, prob, num_agents, num_iterations, num_replicas)
    
    return perf

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Ajusta el número de pruebas según sea necesario

print("Best parameters:", study.best_params)
print("Best performance:", study.best_value)


In [ ]:
import optuna
from optuna.importance import get_param_importances
import pandas as pd
import numpy as np
import time
import multiprocessing
from joblib import Parallel, delayed

# Imports de tu librería
from customhys import benchmark_func as bf
from customhys import metaheuristic as mh
from ml_utils import mittag_leffler  

DATA = None
T_DATA = None
Y_DATA = None


def init_data(data_file):
    global DATA, T_DATA, Y_DATA
    DATA = pd.read_csv(data_file)
    T_DATA = DATA["time"] / DATA["time"].max()
    Y_DATA = DATA["median"] / DATA["median"].max()


def fractional_zener_model(t, params):
    E0, E1, tau, alpha = params
    z = -(t / tau) ** alpha
    ml_values = mittag_leffler(alpha, z)
    return E0 + E1 * ml_values


class P1(bf.BasicProblem):
    def __init__(self, variable_num, min_search_range, max_search_range):
        super().__init__(variable_num)
        self.min_search_range = np.array(min_search_range)
        self.max_search_range = np.array(max_search_range)
        self.global_optimum_solution = 0.0
        self.func_name = 'P1'

    def get_func_val(self, variables, *args):
        y_pred = fractional_zener_model(T_DATA, variables)
        residuals = Y_DATA - y_pred
        mse = np.mean(residuals ** 2)
        mae = np.mean(np.abs(residuals))
        return mse + mae



def evaluate_sequence_performance_all(sequence, prob, num_agents, num_iterations, num_replicas):
    def run_metaheuristic():
        start_t = time.time()
        met = mh.Metaheuristic(prob, sequence, num_agents=num_agents, num_iterations=num_iterations)
        met.run()
        _, f_best = met.get_solution()
        elapsed = time.time() - start_t
        return f_best, elapsed

    num_cores = multiprocessing.cpu_count()
    results_parallel = Parallel(n_jobs=num_cores)(
        delayed(run_metaheuristic)() for _ in range(num_replicas)
    )

    fitness_vals = np.array([r[0] for r in results_parallel], dtype=float)
    times_vals = np.array([r[1] for r in results_parallel], dtype=float)

    fitness_median = np.median(fitness_vals)
    iqr = np.percentile(fitness_vals, 75) - np.percentile(fitness_vals, 25)
    perf_median_iqr = fitness_median + iqr
    avg_time = np.mean(times_vals)

    return fitness_vals, perf_median_iqr, avg_time



data_file = r"C:\Users\A00836756\PycharmProjects\Micro_HH\MG63_2f1_scaled_data.csv"

init_data(data_file)

# Rango min y max para [E0, E1, tau, alpha]
min_range = [0.001, 0.001, 0.001, 0.1]
max_range = [1.5, 1.5, 1.5, 1.0]
fun = P1(variable_num=4, min_search_range=min_range, max_search_range=max_range)
prob = fun.get_formatted_problem()

num_agents = 48
num_iterations = 70
num_replicas = 20

results = []


def objective(trial):
    factor1 = trial.suggest_float("factor1", 0.1, 1.0)
    self_conf1 = trial.suggest_float("self_conf1", 0.1, 4.0)
    swarm_conf1 = trial.suggest_float("swarm_conf1", 0.1, 4.0)
    factor2 = trial.suggest_float("factor2", 0.1, 1.0)
    self_conf2 = trial.suggest_float("self_conf2", 0.1, 4.0)
    swarm_conf2 = trial.suggest_float("swarm_conf2", 0.1, 4.0)
    probability = trial.suggest_float("probability", 0.1, 1.0)
    scale = trial.suggest_float("scale", 0.1, 1.0)

    heur2 = [
        ('swarm_dynamic', {'factor': factor1, 'self_conf': self_conf1, 'swarm_conf': swarm_conf1, 'version': 'constriction', 'distribution': 'uniform'}, 'all'),
        ('swarm_dynamic', {'factor': factor2, 'self_conf': self_conf2, 'swarm_conf': swarm_conf2, 'version': 'inertial', 'distribution': 'uniform'}, 'greedy'),
        ('local_random_walk', {'probability': probability, 'scale': scale, 'distribution': 'gaussian'}, 'greedy')
    ]

    _, perf, _ = evaluate_sequence_performance_all(heur2, prob, num_agents, num_iterations, num_replicas)
    return perf


for i in range(10):
    print(f"rep {i + 1}...")
    
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50)
    
    param_importances = get_param_importances(study)
    
    results.append({
        "replica": i + 1,
        "best_params": study.best_params,
        "best_performance": study.best_value,
        "param_importances": param_importances
    })

df_results = pd.DataFrame(results)

df_importances = pd.DataFrame([result["param_importances"] for result in results])
df_combined = pd.concat([df_results.drop(columns="param_importances"), df_importances], axis=1)

print(df_combined)
output_file = "optuna_results_best_3op.csv"
df_combined.to_csv(output_file, index=False)
print(f"Resultados almacenados en {output_file}")
